### This notebook is Initial submission for the hackathon 
https://zindi.africa/competitions/zimnat-insurance-recommendation-challenge

In [1]:
import pandas as pd
import numpy as np

In [2]:

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils


from keras import optimizers
from keras.layers.normalization import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import Dropout

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
import lightgbm as lgbm
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.model_selection import train_test_split


In [4]:
train_df = pd.read_csv('data/Train_Individual_product_masked.csv')
test_df = pd.read_csv('data/Test.csv')
submissions= pd.read_csv('submissions/SampleSubmission.csv')

In [5]:
submissions.head(2)

,ID X PCODE,Label
0,F86J5PC X P5DA,0
1,F86J5PC X RIBP,0


In [6]:
train_df.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,...,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3,total_bought,target
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,...,0,1,0,0,0,0,0,0,2,RVSZ
1,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,...,0,0,0,0,0,0,0,0,2,K6QO
2,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,...,0,1,0,0,0,0,0,0,2,RVSZ
3,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,...,0,0,0,0,0,0,0,0,2,K6QO
4,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,0,0,...,0,0,0,0,0,0,0,1,3,SOP4


In [7]:
test_df.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,F86J5PC,1/12/2018,M,M,1984,94KC,DZRV,90QI,0,0,...,0,0,0,0,0,0,0,0,0,0
1,H6141K3,1/10/2019,M,M,1996,1X1H,J9SY,90QI,0,0,...,0,0,0,1,0,0,0,0,0,0
2,RBAYUXZ,1/1/2020,F,W,1968,UAOD,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
3,KCBILBQ,1/2/2019,M,M,1989,94KC,2A7I,T4MS,0,0,...,0,0,0,0,0,0,0,0,0,0
4,LSEC1ZJ,1/2/2020,F,M,1982,UAOD,0KID,T4MS,0,0,...,0,0,0,0,0,0,1,0,0,0


### EDA

In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID                        10000 non-null  object
 1   join_date                 9999 non-null   object
 2   sex                       10000 non-null  object
 3   marital_status            10000 non-null  object
 4   birth_year                10000 non-null  int64 
 5   branch_code               10000 non-null  object
 6   occupation_code           10000 non-null  object
 7   occupation_category_code  10000 non-null  object
 8   P5DA                      10000 non-null  int64 
 9   RIBP                      10000 non-null  int64 
 10  8NN1                      10000 non-null  int64 
 11  7POT                      10000 non-null  int64 
 12  66FJ                      10000 non-null  int64 
 13  GYSR                      10000 non-null  int64 
 14  SOP4                   

In [9]:
test_df.join_date.mode()

0    1/5/2018
dtype: object

In [11]:
train_df.ID.nunique()

29132

In [12]:
print(train_df.branch_code.nunique())
print(test_df.branch_code.nunique())
print(len(set(train_df.branch_code.unique()) - set(test_df.branch_code.unique())))
print(len(set(test_df.branch_code.unique()) - set(train_df.branch_code.unique())))

15
15
0
0


In [13]:
print(train_df.occupation_code.nunique())
print(test_df.occupation_code.nunique())
print(len(set(train_df.occupation_code.unique()) - set(test_df.occupation_code.unique())))
print(len(set(test_df.occupation_code.unique()) - set(train_df.occupation_code.unique())))

233
187
55
9


In [14]:
print(train_df.occupation_category_code.nunique())
print(test_df.occupation_category_code.nunique())
print(len(set(train_df.occupation_category_code.unique()) - set(test_df.occupation_category_code.unique())))
print(len(set(test_df.occupation_category_code.unique()) - set(train_df.occupation_category_code.unique())))

6
6
0
0


In [15]:
print(train_df.marital_status.nunique())
print(test_df.marital_status.nunique())
print(len(set(train_df.marital_status.unique()) - set(test_df.marital_status.unique())))
print(len(set(test_df.marital_status.unique()) - set(train_df.marital_status.unique())))

8
8
1
1


In [16]:
len(set(train_df.ID.unique()).intersection(set(test_df.ID.unique())))

0

In [17]:
product_list = ['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9',
                'N2MW', 'AHXO', 'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']

In [18]:
def print_products_info(df):
    user_bought_sum = df[product_list].sum(axis=1)
    print("Total produts bought by all users ",user_bought_sum.sum())
    print("Average number of buys ", user_bought_sum.mean())
    print("Max number of buys ", user_bought_sum.max())
    print("min number of buys ", user_bought_sum.min())
#     print("percentage_disitribution")

In [19]:
print_products_info(train_df)

Total produts bought by all users  95358
Average number of buys  1.4371317046704746
Max number of buys  13
min number of buys  1


In [20]:
print_products_info(test_df)

Total produts bought by all users  12853
Average number of buys  1.2853
Max number of buys  7
min number of buys  1


In [21]:
train_df['total_bought'] = train_df[product_list].sum(axis=1)
test_df['total_bought'] = test_df[product_list].sum(axis=1)

In [25]:
df_train = train_df.copy()

In [40]:
train_df.ID.nunique()

29132

In [41]:
train_df = train_df.groupby('ID', as_index=False).sample(1)

In [43]:
train_df = train_df.reset_index(drop=True)

In [44]:
target = train_df['target']
target_df = pd.get_dummies(target)

### Processing

In [46]:
def get_preprocessed_df(train, test):
    print("Train shape :", train.shape)
    print("Test shape :", test.shape)
    df = train.drop('target', axis=1).append(test, ignore_index=True)
    df['sex'] = df.sex.apply(lambda x: 1 if x=='F' else 0)
    df = pd.concat([df, pd.get_dummies(df.marital_status, prefix='marital_status')], axis=1)
    df = pd.concat([df, pd.get_dummies(df.occupation_category_code, prefix='occupation_category_code')], axis=1)
    df = pd.concat([df, pd.get_dummies(df.branch_code, prefix='branch_code')], axis=1)
    df['join_date'] = pd.to_datetime(df.join_date)
    df.join_date = df.join_date.fillna(df.join_date.mode()[0])
    print(df.join_date.isna().value_counts())
    df['join_date_month'] = df.join_date.dt.month
    df['join_day_of_the_month'] = df.join_date.dt.day
    df['join_year'] = df.join_date.dt.year
    df = df.drop(columns=['marital_status', 'occupation_category_code', 'branch_code', 'occupation_code',
                          'join_date'], axis=1)
    print("All data columns list: ", df.columns)
    print("All data shape :", df.shape)
    return df

In [47]:
all_data = get_preprocessed_df(train_df, test_df)

Train shape : (29132, 31)
Test shape : (10000, 30)
False    39132
Name: join_date, dtype: int64
All data columns list:  Index(['ID', 'sex', 'birth_year', 'P5DA', 'RIBP', '8NN1', '7POT', '66FJ',
       'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO', 'BSTQ', 'FM3X',
       'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3', 'total_bought',
       'marital_status_D', 'marital_status_F', 'marital_status_M',
       'marital_status_P', 'marital_status_R', 'marital_status_S',
       'marital_status_U', 'marital_status_W', 'marital_status_f',
       'occupation_category_code_56SI', 'occupation_category_code_90QI',
       'occupation_category_code_AHH5', 'occupation_category_code_JD7X',
       'occupation_category_code_L44T', 'occupation_category_code_T4MS',
       'branch_code_1X1H', 'branch_code_30H5', 'branch_code_49BM',
       'branch_code_748L', 'branch_code_94KC', 'branch_code_9F9T',
       'branch_code_BOAS', 'branch_code_E5SW', 'branch_code_EU3L',
       'branch_code_O4JC', 

In [48]:
all_data.dtypes.value_counts()

uint8     30
int64     27
object     1
dtype: int64

In [49]:
train_data = all_data[:29132]
pred_data = all_data[29132:].reset_index(drop=True)

In [51]:
(train_data.ID==train_df.ID).value_counts()

True    29132
Name: ID, dtype: int64

In [52]:
(pred_data.ID==test_df.ID).value_counts()

True    10000
Name: ID, dtype: int64

In [53]:
train_data.join_date_month.isna().value_counts()

False    29132
Name: join_date_month, dtype: int64

In [54]:
def create_id_pcode(model, group, model_type):
    if model_type=='DL':
        classes_ser = pd.Series(list(target_df.columns))
    else:
        classes_ser = pd.Series(model.classes_)
    classes_ser.index = group.index
    group['ID X PCODE'] = group['ID'] + ' X ' + classes_ser
    return group

def save_predictions(model, predict_data, file_name_prefix, model_type='ML', round_off=5):
    if model_type!='DL':
        predictions = model.predict_proba(predict_data.drop('ID', axis=1))
    else:
        predictions = model.predict_proba(predict_data)
    #predictions = np.round(predictions, decimals=round_off)
    sub_df = pd.DataFrame({'ID':pred_data['ID'], 'PCODE' :predictions.tolist()})
    temp = sub_df.explode('PCODE').reset_index(drop=True)
    temp = temp.groupby('ID', as_index=False).apply(lambda x: create_id_pcode(model, x, model_type)
                                                   ).rename(columns = {'PCODE':'Label'}
                                                           )[['ID X PCODE', 'Label']]
    temp['Label'] = temp['Label'].astype(float)
    # return temp
    temp.to_csv('submissions/' + file_name_prefix + '_submission.csv', index=False, float_format='%.7f')

### Model Building and submissions!! Yayy!!

In [55]:
kfold = KFold(n_splits=5, shuffle=True)

In [56]:
from sklearn.preprocessing import  StandardScaler
scaler = StandardScaler()

X= scaler.fit_transform(train_data.drop('ID', axis=1))

In [57]:
X_pred = scaler.transform(pred_data.drop('ID', axis=1))

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, target_df, test_size=0.25)

In [59]:
import xgboost

In [60]:
batch = 256
lr = 0.009
adam = optimizers.Adam(lr)
leaky_relu_alpha =0.1

In [331]:
# create model
model = Sequential()
model.add(Dense(228, input_dim=57, kernel_initializer='normal'))
#model.add(BatchNormalization())
model.add(LeakyReLU(alpha=leaky_relu_alpha))
model.add(Dropout(0.2))

model.add(Dense(114, kernel_initializer='normal'))
#model.add(BatchNormalization())
model.add(LeakyReLU(alpha=leaky_relu_alpha))
model.add(Dropout(0.2))

model.add(Dense(42, activation='relu', kernel_initializer='normal'))
model.add(LeakyReLU(alpha=leaky_relu_alpha))
#model.add(Dropout(0.2))
model.add(Dense(21, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics=['accuracy'])
model.summary()

Model: "sequential_73"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_218 (Dense)            (None, 228)               13224     
_________________________________________________________________
leaky_re_lu_115 (LeakyReLU)  (None, 228)               0         
_________________________________________________________________
dropout_68 (Dropout)         (None, 228)               0         
_________________________________________________________________
dense_219 (Dense)            (None, 114)               26106     
_________________________________________________________________
leaky_re_lu_116 (LeakyReLU)  (None, 114)               0         
_________________________________________________________________
dropout_69 (Dropout)         (None, 114)               0         
_________________________________________________________________
dense_220 (Dense)            (None, 42)              

In [336]:
callback = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5,
                                        mode='auto', restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=3,
                                            min_lr=0.00001)]

In [337]:
# estimator = KerasClassifier(build_fn=baseline_model

In [338]:
# results = cross_val_score(estimator, train_data.drop('ID', axis=1), target_df, cv=kfold, verbose=2)
# print("dl: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
# results
model_history = model.fit(X_train, y_train,
                          validation_data=(X_test,y_test),
                          epochs=200, batch_size=batch, verbose=1,
                          callbacks=callback)

Train on 49764 samples, validate on 16589 samples
Epoch 1/200
49764/49764 [==============================] - 1s 25us/step - loss: 0.3879 - accuracy: 0.8761 - val_loss: 0.4151 - val_accuracy: 0.8691
Epoch 2/200
49764/49764 [==============================] - 1s 25us/step - loss: 0.3887 - accuracy: 0.8765 - val_loss: 0.4151 - val_accuracy: 0.8691
Epoch 3/200
49764/49764 [==============================] - 1s 25us/step - loss: 0.3881 - accuracy: 0.8756 - val_loss: 0.4151 - val_accuracy: 0.8688
Epoch 4/200
49764/49764 [==============================] - 1s 25us/step - loss: 0.3884 - accuracy: 0.8765 - val_loss: 0.4151 - val_accuracy: 0.8691
Epoch 5/200
49764/49764 [==============================] - 1s 24us/step - loss: 0.3890 - accuracy: 0.8752 - val_loss: 0.4151 - val_accuracy: 0.8691
Epoch 6/200
49764/49764 [==============================] - 1s 25us/step - loss: 0.3888 - accuracy: 0.8761 - val_loss: 0.4150 - val_accuracy: 0.8693
Epoch 7/200
49764/49764 [==============================] - 1s 

In [341]:
model_history = model.fit(X, target_df,
                          validation_data=(X_test,y_test),
                          epochs=20, batch_size=batch, verbose=1,
                          callbacks=callback)

Train on 66353 samples, validate on 16589 samples
Epoch 1/20
66353/66353 [==============================] - 2s 26us/step - loss: 0.4016 - accuracy: 0.8722 - val_loss: 0.4138 - val_accuracy: 0.8694
Epoch 2/20
66353/66353 [==============================] - 2s 24us/step - loss: 0.3997 - accuracy: 0.8730 - val_loss: 0.4131 - val_accuracy: 0.8696
Epoch 3/20
66353/66353 [==============================] - 2s 26us/step - loss: 0.4003 - accuracy: 0.8729 - val_loss: 0.4125 - val_accuracy: 0.8700
Epoch 4/20
66353/66353 [==============================] - 2s 24us/step - loss: 0.4019 - accuracy: 0.8728 - val_loss: 0.4119 - val_accuracy: 0.8701
Epoch 5/20
66353/66353 [==============================] - 2s 25us/step - loss: 0.4014 - accuracy: 0.8734 - val_loss: 0.4116 - val_accuracy: 0.8701
Epoch 6/20
66353/66353 [==============================] - 2s 24us/step - loss: 0.3998 - accuracy: 0.8719 - val_loss: 0.4109 - val_accuracy: 0.8702
Epoch 7/20
66353/66353 [==============================] - 2s 25us/st

In [344]:
model.evaluate(X_test,y_test)

16589/16589 [==============================] - 1s 70us/step


[0.40653494059629913, 0.8710591197013855]

In [345]:
model.predict_proba(X_pred)

array([[2.94995825e-05, 8.14760915e-06, 9.89021100e-06, ...,
        1.13811046e-04, 3.59683622e-06, 8.96496495e-05],
       [2.21499498e-03, 1.11731053e-04, 1.13448899e-04, ...,
        1.21669853e-02, 1.68536149e-03, 2.77323904e-03],
       [1.79893832e-04, 2.51625152e-05, 2.12652412e-05, ...,
        4.84824326e-04, 9.47586086e-05, 1.33988244e-04],
       ...,
       [3.06474579e-09, 8.39255998e-10, 1.81320875e-10, ...,
        1.47882142e-08, 9.99998927e-01, 9.03474795e-10],
       [6.37342855e-02, 4.32902901e-03, 1.29848840e-02, ...,
        3.42637360e-01, 1.93970263e-01, 6.77727982e-02],
       [4.26845436e-05, 1.88561359e-07, 3.22780966e-06, ...,
        6.72417373e-05, 9.99832630e-01, 1.47404153e-05]], dtype=float32)

In [ ]:
pd.DataFrame({'ID':pred_data['ID'], 
              'PCODE' : estimator.predict_proba(pred_data.drop('ID', axis=1)).tolist()
             }).explode('PCODE').reset_index(drop=True)

In [ ]:
estimator.classes_

In [352]:
save_predictions(model, X_pred, 'dl_Dense-228-114-42-21_Lrelu1_LR009_batch256', model_type='DL')#.info()

In [61]:
lgbmc = lgbm.LGBMClassifier(num_leaves=10, n_estimators=50)

In [62]:
xgbc = xgboost.XGBClassifier(num_leaves=10, n_estimators=50, max_depth=21, learning_rate=0.5)

In [63]:
rfc = RandomForestClassifier( max_depth=60)

In [68]:
skfold = StratifiedKFold(n_splits=5, shuffle=True)

In [70]:
results = cross_val_score(lgbmc, X, target, cv=skfold, verbose=2)#, scoring='neg_log_loss')
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
results

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/lavanyamk/venv_dedupe/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[CV]  ................................................................
[CV] ................................................. , total=   7.5s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.2s remaining:    0.0s


[CV] ................................................. , total=  11.0s
[CV]  ................................................................
[CV] ................................................. , total=  12.0s
[CV]  ................................................................
[CV] ................................................. , total=  19.4s
[CV]  ................................................................
[CV] ................................................. , total=   8.8s
Baseline: 86.99% (0.28%)


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   59.5s finished


array([0.87073547, 0.87251201, 0.87131091, 0.87068521, 0.86443299])

In [ ]:
lgbmc.fit(X, target)

In [ ]:
results = cross_val_score(xgbc, X, target, cv=skfold, verbose=2, scoring='neg_log_loss')
print("xgbc: %.2f%% (%.2f%%)" % (results.mean(), results.std()))
results

In [142]:
xgbc.fit(train_data.drop('ID', axis=1), target)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=50, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [106]:
save_predictions(lgbmc, pred_data, 'lgbmc_num_leaves_50-n_estimators_400')#.info()

In [145]:
save_predictions(xgbc, pred_data, 'xgbc_n_estimators_50')#.info()